In [1]:
import numpy as np
import sympy as sp
from manipulator_graph import frame_inference as fi
from manipulator_graph import URDFParser
from manipulator_graph import sort_joints
from manipulator_graph import line_geometry as lg
from manipulator_graph import parameter_inference as pi

In [2]:
parser = URDFParser('./kr210.urdf')
joints = [j for j in parser.joints if 'gripper' not in j.name]
joints = sort_joints(joints)
axis = [j.axis for j in joints]
position = [j.position for j in joints]

In [3]:
# create joint line plot from
def joint_axis(joints):
    for j in joints:
        if j.axis is None:
            continue
        axis = lg.Line(k=j.axis, )
        

def homogenous_transfrom(rpy, origin):
    x_rot = sp.rot_axis1(rpy[0])
    y_rot = sp.rot_axis2(rpy[1])
    z_rot = sp.rot_axis3(rpy[2])
    origin = sp.Matrix(origin)
    rot = z_rot * y_rot * x_rot
    rot = rot.row_join(origin)
    rot = rot.col_join(sp.Matrix([[0, 0, 0, 1]]))
    rot = sp.simplify(rot)
    rot = np.array(rot.tolist(), dtype=np.float64)
    return rot


In [4]:
def inference_frames(joints):
    frame = np.eye(4)
    ret = []
    for j in joints:
        frame = np.matmul(frame, homogenous_transfrom(
                j.orientation, j.position))
        ret.append(frame)
    return ret

In [5]:
ret = inference_frames(joints[1:])

In [6]:
def absolute_axis(axes, frames):
    assert len(axes) == len(frames)
    ret = []
    for a, f in zip(axes, frames):
        line = lg.Line(k=np.matmul(f[:3, :3], a), p=f[:3, -1])
        ret.append(line)
    return ret

In [7]:
abs_axis = absolute_axis(axis[1:], ret)

In [8]:
dh_frames = fi.assign_dh_frames(abs_axis)

Line(p=array([ 0.35,  0.  ,  0.75]), k=array([ 0.,  1.,  0.])) Line(p=array([ 0.35,  0.  ,  2.  ]), k=array([ 0.,  1.,  0.]))
[ 1. -0. -0.] [ 0.    0.    0.75]


In [9]:
dh_params = pi.inference_dh_parameters(dh_frames)

In [25]:
def dh_frame_to_homogenous(frame):
    y = np.cross(frame.z, frame.x)
    mat = np.array([frame.x, y, frame.z]).transpose()
    mat = np.hstack((mat, frame.o.reshape(-1, 1)))
    mat = np.vstack((mat, (0, 0, 0, 1)))
    return mat

In [27]:
dh_frame_to_homogenous(dh_frames[1])

array([[ 1.  ,  0.  ,  0.  ,  0.  ],
       [-0.  ,  1.  ,  0.  ,  0.  ],
       [-0.  , -0.  ,  1.  ,  0.75],
       [ 0.  ,  0.  ,  0.  ,  1.  ]])

In [28]:
dh_frames[1]

DHFrame(z=array([ 0.,  0.,  1.]), x=array([ 1., -0., -0.]), o=array([ 0.  ,  0.  ,  0.75]))

In [34]:
joints[2]

Joint(name=u'joint_2', position=array([ 0.35,  0.  ,  0.42]), orientation=array([ 0.,  0.,  0.]), axis=array([ 0.,  1.,  0.]), type=u'revolute', parent=Link(name=u'link_1', position=array([ 0. ,  0. ,  0.4]), orientation=array([ 0.,  0.,  0.]), mass=138.0, inertia=array([[ 30.,   0.,   0.],
       [  0.,  50.,   0.],
       [  0.,   0.,  50.]])), child=Link(name=u'link_2', position=array([ 0.     ,  0.     ,  0.44751]), orientation=array([ 0.,  0.,  0.]), mass=95.0, inertia=array([[ 40.,   0.,   0.],
       [  0.,  40.,   0.],
       [  0.,   0.,  10.]])), limits=None)

In [35]:
dh_frames[1]

DHFrame(z=array([ 0.,  0.,  1.]), x=array([ 1., -0., -0.]), o=array([ 0.  ,  0.  ,  0.75]))

In [36]:
abs_axis[1]

Line(p=array([ 0.35,  0.  ,  0.75]), k=array([ 0.,  1.,  0.]))

In [37]:
joints[1]

Joint(name=u'joint_1', position=array([ 0.  ,  0.  ,  0.33]), orientation=array([ 0.,  0.,  0.]), axis=array([ 0.,  0.,  1.]), type=u'revolute', parent=Link(name=u'base_link', position=array([ 0.,  0.,  0.]), orientation=array([ 0.,  0.,  0.]), mass=150.0, inertia=array([[  60.,    0.,    0.],
       [   0.,   70.,    0.],
       [   0.,    0.,  100.]])), child=Link(name=u'link_1', position=array([ 0. ,  0. ,  0.4]), orientation=array([ 0.,  0.,  0.]), mass=138.0, inertia=array([[ 30.,   0.,   0.],
       [  0.,  50.,   0.],
       [  0.,   0.,  50.]])), limits=None)

In [42]:
dh_frames[2]

DHFrame(z=array([ 0.,  1.,  0.]), x=array([-0., -0.,  1.]), o=array([ 0.  ,  0.  ,  0.75]))

In [39]:
len(dh_frames)

7

In [43]:
len(joints)

7

In [45]:
joints[0]

Joint(name=u'fixed_base_joint', position=array([ 0.,  0.,  0.]), orientation=array([ 0.,  0.,  0.]), axis=None, type=u'fixed', parent=Link(name=u'base_footprint', position=array([ 0.,  0.,  0.]), orientation=array([ 0.,  0.,  0.]), mass=None, inertia=None), child=Link(name=u'base_link', position=array([ 0.,  0.,  0.]), orientation=array([ 0.,  0.,  0.]), mass=150.0, inertia=array([[  60.,    0.,    0.],
       [   0.,   70.,    0.],
       [   0.,    0.,  100.]])), limits=None)

In [9]:
dh_frames[2]

DHFrame(z=array([ 0.,  1.,  0.]), x=array([-0., -0.,  1.]), o=array([ 0.35,  0.  ,  0.75]))

In [10]:
dh_params

(DHValue(a=0.0, alpha=0, d=0.0, theta=0),
 DHValue(a=0.34999999999999998, alpha=-1.5707963267948966, d=0.0, theta=-1.5707963267948966),
 DHValue(a=1.25, alpha=0, d=0.0, theta=3.141592653589793),
 DHValue(a=0.053999999999999999, alpha=1.5707963267948966, d=1.5, theta=3.141592653589793),
 DHValue(a=0.0, alpha=1.5707963267948966, d=0.0, theta=3.141592653589793),
 DHValue(a=-0.0, alpha=1.5707963267948966, d=0.0, theta=0))